# **AAL Australia: 4Q Sales Report**


## **1. Data Wrangling**


### **1.1 Data Inspection**

_It's the process of getting familiar with the data in order to identify quality and structure issues._


In [27]:
import pandas as pd

df = pd.read_csv("./Data/AAL_Q4-2020_Sales.csv")

print("First 5 rows of the DataFrame:")
print(df.head(5))

print(f"\nLast 5 rows of the DataFrame:")
print(df.tail(5))

print("\nDataFrame Summary")
print(df.info())

has_null = df.isnull().values.any()
print("\nDoes the DataFrame have any null value?:", has_null)

number_of_duplicates = df.duplicated().sum()
print(f"\nNumber of duplicates: {number_of_duplicates}")

print("\nList of numerical columns:")
numerical_columns = df.select_dtypes(include=["number"]).columns
print(numerical_columns)

print("\nList of categorical columns:")
categorical_columns = df.select_dtypes(exclude=["number"]).columns
print(categorical_columns)

First 5 rows of the DataFrame:
         Date        Time State     Group  Unit  Sales
0  1-Oct-2020     Morning    WA      Kids     8  20000
1  1-Oct-2020     Morning    WA       Men     8  20000
2  1-Oct-2020     Morning    WA     Women     4  10000
3  1-Oct-2020     Morning    WA   Seniors    15  37500
4  1-Oct-2020   Afternoon    WA      Kids     3   7500

Last 5 rows of the DataFrame:
             Date        Time State     Group  Unit  Sales
7555  30-Dec-2020   Afternoon   TAS   Seniors    14  35000
7556  30-Dec-2020     Evening   TAS      Kids    15  37500
7557  30-Dec-2020     Evening   TAS       Men    15  37500
7558  30-Dec-2020     Evening   TAS     Women    11  27500
7559  30-Dec-2020     Evening   TAS   Seniors    13  32500

DataFrame Summary
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7560 entries, 0 to 7559
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    7560 non-null   object
 1   Time    7560 non

### **1.2 Data Cleaning**

_It's the process of handling missing data, removing duplicates, converting data types, trimming whitespace, correcting inconsistencies, standardizing formats, dealing with outliers, and validating accuracy._


In [28]:
# Handling duplicates:
if number_of_duplicates > 0:
    df = df.drop_duplicates(keep="first")

# Handling missing data for numerical columns:
for column in numerical_columns:
    if df[column].isnull().any():
        column_median = df[column].median()
        df[column] = df[column].fillna(column_median)

# Handling missing data for categorical columns:
for column in categorical_columns:
    if df[column].isnull().any():
        df[column] = df[column].fillna("Unknown")


# Identify and correct misspelled words and unnecessary whitespace:
for column in categorical_columns:
    if column != "Date":
        df[column] = df[column].str.strip()
        unique_values = df[column].unique()
        print(f"{column} unique values: {unique_values}")

# Handling Outliers:
SALES_COLUMN = "Sales"

q1 = df[SALES_COLUMN].quantile(0.25)
q3 = df[SALES_COLUMN].quantile(0.75)
iqr = q3 - q1

lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# Tagging all the 'sales outliers' with True or False
condition = (df[SALES_COLUMN] >= lower_bound) & (df[SALES_COLUMN] <= upper_bound)
df["Sales_Outlier"] = ~condition

Time unique values: ['Morning' 'Afternoon' 'Evening']
State unique values: ['WA' 'NT' 'SA' 'VIC' 'QLD' 'NSW' 'TAS']
Group unique values: ['Kids' 'Men' 'Women' 'Seniors']


### **1.3 Data Transformation**

_It involves converting data from its original form into a format that is more suitable for analysis._


In [29]:
from pandas import DataFrame
from sklearn.preprocessing import MinMaxScaler

# Transforming Date column to YYYY-MM-DD:
DATE_COLUMN = "Date"
df[DATE_COLUMN] = pd.to_datetime(df[DATE_COLUMN], format="%d-%b-%Y")


# Encoding categorical data into numerical:
def encode(df: DataFrame, original_column: str, new_column: str):
    df[new_column], _unique = pd.factorize(df[original_column])


ORIGINAL_COLUMNS = ["Time", "State", "Group"]
NEW_COLUMNS = ["Numerical_Time", "Numerical_State", "Numerical_Group"]

for original_column, new_column in zip(ORIGINAL_COLUMNS, NEW_COLUMNS):
    encode(df, original_column, new_column)


# Binning Sales column to create Sales_Range:
SALES_RANGE = "Sales_Range"

min = df[SALES_COLUMN].min()
q2 = df[SALES_COLUMN].quantile(0.5)
max = df[SALES_COLUMN].max()

bin_edges = [min, q1, q2, q3, max]
bin_labels = ["0-25%", "25%-50%", "50%-75%", "75%-100%"]

df[SALES_RANGE] = pd.cut(
    df[SALES_COLUMN], bins=bin_edges, labels=bin_labels, include_lowest=True
)

# Normalization
cleaned_df = df.copy(deep=True)
normalized_df = cleaned_df.copy(deep=True)
new_numerical_columns = normalized_df.select_dtypes(include=["number"]).columns

SCALER = MinMaxScaler()

normalized_df[new_numerical_columns] = SCALER.fit_transform(
    normalized_df[new_numerical_columns]
)

## **2. Data Saving**

_It involves creating a new DataFrame with the wrangled data and save it into a new csv file._


In [30]:
cleaned_df.to_csv("./Data/AAL_Q4-2020_Sales_Cleaned.csv", index=False)
normalized_df.to_csv("./Data/AAL_Q4-2020_Sales_Cleaned_and_Normalized.csv", index=False)

## **3. Data Analysis**


### **3.1 Descriptive Statistical Analysis**


In [32]:
cleaned_stats = cleaned_df['Sales'].describe().round()
print(cleaned_stats)

count      7560.0
mean      45014.0
std       32254.0
min        5000.0
25%       20000.0
50%       35000.0
75%       65000.0
max      162500.0
Name: Sales, dtype: float64
